In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math

# Using `widget` here instead of `notebook` or `inline` to allow for dynamic updating in Jupter Lab 
import matplotlib.pyplot as plt

# Progress bar 
from tqdm import tqdm_notebook, tnrange

In [2]:
import datetime
now = datetime.datetime.now().isoformat()

In [3]:
torch.manual_seed(1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"device:\n{device}\n")

# LSTM 

In [4]:
%%time
%matplotlib widget

EPOCHS = 1000
NB_EPOCHS_UPDATE_UI = 25

period_samples = 100
periods_count = 2

inputs = torch.tensor([[[math.sin(2 * math.pi * x / float(period_samples))]] for x in
                       range(period_samples * periods_count)])
# print(inputs.size())

# LST model
model = nn.LSTM(input_size=1, hidden_size=1)
model.to(device)

loss_function = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

X = inputs[:-1]
X_gpu = X.to(device)

Y = inputs[1:]
Y_gpu = Y.to(device)

h_0 = torch.zeros(1, 1, 1, dtype=torch.float32).to(device)
c_0 = torch.zeros(1, 1, 1, dtype=torch.float32).to(device)

losses = []

def update_plot(fig, ax_loss, ax_fit, losses, X, out_values):
    ax_loss.clear()
    ax_loss.plot(losses)

    ax_fit.clear()
    ax_fit.scatter(range(period_samples * periods_count - 1), X.numpy(), color='red', alpha=0.2, label="X")
    ax_fit.scatter(range(period_samples * periods_count - 1), out_values.numpy(), color='blue', alpha=0.2, label="out")
    ax_fit.legend()

    fig.canvas.draw()


# Progress bar: tnrange, is a wrapper for tqdm_notebook(range(),xxx)
# See https://pypi.org/project/tqdm/#manual
with tnrange(EPOCHS, desc="epoch") as t:
    fig, ax = plt.subplots(2,1)
    ax_loss = ax[0]
    ax_fit = ax[1]

    for epoch in t:
        # Forward
        out_values, _ = model(X_gpu, (h_0, c_0))

        # Compute the loss, gradients, and update the parameters by calling optimizer.step()
        optimizer.zero_grad()
        loss = loss_function(out_values, Y_gpu)
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        t.set_postfix(loss=loss.item())

        if epoch % NB_EPOCHS_UPDATE_UI == 0:
            update_plot(fig, ax_loss, ax_fit, losses, X, out_values.cpu().detach())
            
    update_plot(fig, ax_loss, ax_fit, losses, X, out_values.cpu().detach())


FigureCanvasNbAgg()


CPU times: user 20.8 s, sys: 12.3 s, total: 33.1 s
Wall time: 19.3 s


In [5]:
X_range = [i for i in range(200)]
Y_hat = []

last_prediction = torch.zeros(1, 1, 1, dtype=torch.float32).to(device)
hidden = (torch.zeros(1, 1, 1, dtype=torch.float32).to(device), torch.zeros(1, 1, 1, dtype=torch.float32).to(device))

for x in X_range:
    out, hidden = model(last_prediction, hidden)
    last_prediction = out
    
    Y_hat.append(out.item())

fig, ax = plt.subplots()
ax.plot(Y_hat, 'bo', alpha=0.2)
ax.plot(inputs.squeeze().numpy(), 'ro', alpha=0.2)
fig.show()

FigureCanvasNbAgg()

# LSTMCell

In [6]:
%%time

ax_loss = None
ax_fit = None


EPOCHS = 1000
NB_EPOCHS_UPDATE_UI = 25

period_samples = 100
periods_count = 2
inputs = torch.tensor([[[math.sin(2 * math.pi * x / float(period_samples))]] for x in
                       range(period_samples * periods_count)], dtype=torch.float32)
print(inputs.size())

# LST model
model = nn.LSTMCell(input_size=1, hidden_size=1)
model.to(device)

loss_function = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

X = inputs[:-1]
X_gpu = X.to(device)

Y = inputs[1:]
Y_gpu = Y.to(device)

losses = []

def update_plot(fig, ax_loss, ax_fit, losses, X, out_values):
    pass
    ax_loss.clear()
    ax_loss.plot(losses)

    ax_fit.clear()
    ax_fit.scatter(range(period_samples * periods_count - 1), X.numpy(), color='red', alpha=0.2, label="X")
    ax_fit.scatter(range(period_samples * periods_count - 1), out_values.numpy(), color='blue', alpha=0.2, label="out")
    ax_fit.legend()

    fig.canvas.draw()

    
# Progress bar: tnrange, is a wrapper for tqdm_notebook(range(),xxx)
# See https://pypi.org/project/tqdm/#manual
with tnrange(EPOCHS, desc="epoch") as progress:
    fig2, ax2 = plt.subplots(2,1)
    ax_loss = ax2[0]
    ax_fit = ax2[1]
    
    for epoch in progress:
        h_0 = torch.zeros(1, 1, dtype=torch.float32).to(device)
        c_0 = torch.zeros(1, 1, dtype=torch.float32).to(device)

        h_1 = h_0
        c_1 = c_0
        
        out_values = torch.zeros(Y.size()[0], 1, 1, dtype=torch.float32).to(device)
        
        # Forward
        for i, x in enumerate(X_gpu):
            h_1, c_1 = model(x, (h_1, c_1))
            out_values[i].add_(h_1)

        # Compute the loss, gradients, and update the parameters by calling optimizer.step()
        optimizer.zero_grad()
        loss = loss_function(out_values, Y_gpu)
        loss.backward()
        optimizer.step()

        losses.append(loss.item())           
        progress.set_postfix(loss=loss.item())
        if epoch % NB_EPOCHS_UPDATE_UI == 0:
            update_plot(fig2, ax_loss, ax_fit, losses, X, out_values.cpu().detach())
            
    update_plot(fig2, ax_loss, ax_fit, losses, X, out_values.cpu().detach())

torch.Size([200, 1, 1])


FigureCanvasNbAgg()


CPU times: user 1min 50s, sys: 12 s, total: 2min 2s
Wall time: 1min 47s


In [9]:
X_range = [i for i in range(200)]
Y_hat = []

last_prediction = torch.zeros(1, 1, dtype=torch.float32).to(device)
h_0 = torch.zeros(1, 1, dtype=torch.float32).to(device)
c_0 = torch.zeros(1, 1, dtype=torch.float32).to(device)

h_1 = h_0
c_1 = c_0

for x in X_range:
    h_1, c_1 = model(last_prediction, (h_1, c_1))
    last_prediction = h_1
    
    Y_hat.append(h_1.item())

fix, ax = plt.subplots()
ax.plot(Y_hat, 'bo', alpha=0.2)
ax.plot(inputs.squeeze().numpy(), 'ro', alpha=0.2)
fig.show()

FigureCanvasNbAgg()